In [50]:
import csv
''' Depois de tratar o texto, e guardar os csv's em (letras.csv), (nutricao.csv)
    Podemos de fato começar a aplicar o método do simplex para determinar as melhores dietas'''

nomes = ['HENRIQUE','LUCAS','RAFAEL'] #nossos nomes

#nutrientes da que nossa dieta precisa satisfazer 
nutrientes_minimos = ['Caloria','Calcio','VitaminaA','Riboflavina','AcidoAscorbico'] 

dieta_prodx_por_letra = dict() #dicionario de letra para dieta e prodx
with open('letras.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dieta_prodx_por_letra[row['Letra']] = (float(row['Dieta']),float(row['ProdutoX']))

Vamos definir agora nosso problema de minimização!

Se gostaríamos de reduzir o custo, nossas variáveis serão todas $x_i$, onde $x_i$ significa quantos dólares gastamos no alimento $i$.
Como o médico nos passou quantidades mínimas para suprir de nutrientes, essas quantidades formarão nossa matriz de restrição.

Teremos algo do tipo:
$$
    x_1c_1 + x_2c_2 + ... + x_nc_n \geq c\\
    x_1p_1 + x_2p_2 + ... + x_np_n \geq p\\
    x_1v_1 + x_2v_2 + ... + x_nv_n \geq v\\
    x_1r_1 + x_2r_2 + ... + x_nr_n \geq r\\
    x_1a_1 + x_2a_2 + ... + x_na_n \geq a\\
$$

Onde $c_i$ é quanto de calórias o alimento $i$ dá por unidade de dolar gasto e $c$ é a quantidade mínima de calorias que o médico nos recomendou. (Fiz a mesma coisa para os outros nutrientes)

Como isso não está exatamente na forma padrão, teremos que multiplicar por -1 todas as linhas, de forma a tornar todo $\geq$ em $\leq$.

In [51]:
''' ideia 
M = [ [c1,c2,c3,...,cn],
      [p1,p2,p3,...,pn],
      ...
    ]
'''
alimentos = []
with open('nutricao.csv','r',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        alimentos.append(row)

Depois de conseguir resultados estranhos, percebi que as dimensões passadas pelo médico e pelas tabelas estão em dimensões diferentes:
- Dimensão Dieta: **$10^2$ calorias, $10^{-2}$ gramas, $10^2$ unidades internacionais, $10^{-1}$ miligramas e miligrama.**
- Dimensões Produto X: **$10^3$ calorias/dólar, $10^{−1}$ gramas/dólar, $10^3$ unidades internacionais/dólar, $10^{−1}$ miligramas/dólar, miligramas/dólar.**

In [61]:
matriz_sem_prod_x = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos]
    matriz_sem_prod_x.append(linha_nutri)


In [65]:
#com a matriz de restrição sem o produto X pronta, já estamos quase finalizados, basta descobrir quanto produto X nos dá e o B para cada nome!
from copy import deepcopy
import numpy as np
import scipy.optimize

conversao_dieta = [10**-1, 10**-2, 10**-1, 10**-1, 1] # para conversão da restrição
conversao_prod_x = [1,10**-1,1,10**-1,1] # para conversao do produto x

def matriz(nome:str):
    m = deepcopy(matriz_sem_prod_x)
    for i in range(5): #são cinco nutrientes
        if i < len(nome):
            letra = nome[i].upper()
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)

def restricoes(nome):
    r = []
    for i in range(5):
        if i < len(nome):
            letra = nome[i].upper()
            val = float(dieta_prodx_por_letra[letra][0]) * conversao_dieta[i]
            r.append(val)
        else:
            r.append(0.)
    return np.array(r)



def dieta(nome:str,method:str):
    m = -matriz(nome)
    r = -restricoes(nome)
    c = np.ones(len(alimentos) + 1) # esse é o funcional linear que será minizado
    return scipy.optimize.linprog(c=c,A_ub=m,b_ub=r,method=method)

In [63]:
matriz("henrique")

matrix([[4.47e+01, 3.60e+01, 8.40e+00, 2.06e+01, 7.40e+00, 1.57e+01,
         4.17e+01, 2.20e+00, 4.40e+00, 5.80e+00, 2.40e+00, 2.60e+00,
         5.80e+00, 1.43e+01, 1.10e+00, 9.60e+00, 8.50e+00, 1.28e+01,
         1.74e+01, 2.69e+01, 6.90e+01],
        [2.00e+00, 1.70e+00, 1.51e+01, 6.00e-01, 1.64e+01, 1.00e+00,
         0.00e+00, 2.00e-01, 3.00e-01, 6.80e+00, 3.70e+00, 4.00e+00,
         3.80e+00, 1.80e+00, 0.00e+00, 2.70e+00, 1.70e+00, 2.50e+00,
         3.70e+00, 1.14e+01, 8.20e+00],
        [3.65e+02, 9.90e+01, 9.00e+00, 6.00e+00, 1.90e+01, 4.80e+01,
         0.00e+00, 1.39e+02, 3.70e+01, 4.50e+01, 8.00e+01, 3.60e+01,
         5.90e+01, 1.18e+02, 1.38e+02, 5.40e+01, 1.73e+02, 1.54e+02,
         4.59e+02, 7.92e+02, 9.10e+01],
        [5.54e+01, 1.74e+01, 3.00e+00, 2.00e-01, 8.00e-01, 9.60e+00,
         0.00e+00, 6.40e+00, 1.82e+01, 1.00e+00, 4.30e+00, 9.00e+00,
         4.70e+00, 2.94e+01, 5.70e+00, 8.40e+00, 1.20e+00, 3.90e+00,
         2.69e+01, 3.84e+01, 6.70e+00],
        [4.4

In [64]:
print(restricoes("henrique"))

[ 3.    0.39  0.6   4.7  65.  ]


In [83]:
minha_dieta = dieta('henrique','highs')
vet = minha_dieta.x
vet.shape = (21,1)
M = matriz('henrique')

M*vet #vamos printar o valor nutricional da minha dieta (está satisfazendo o pedido!)

matrix([[ 4.3088814 ],
        [ 0.39      ],
        [39.10514914],
        [ 4.7       ],
        [65.        ]])

In [86]:
minha_dieta

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.1511039678368966
              x: [[ 6.014e-02]
                  [ 0.000e+00]
                  ...
                  [ 1.719e-02]
                  [ 0.000e+00]]
            nit: 3
          lower:  residual: [ 6.014e-02  0.000e+00 ...  1.719e-02
                              0.000e+00]
                 marginals: [ 0.000e+00  6.995e-01 ...  0.000e+00
                              5.120e-01]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.309e+00  0.000e+00  3.851e+01  0.000e+00
                              0.000e+00]
                 marginals: [-0.000e+00 -4.849e-02 -0.000e+00 -2.611e-04

In [84]:
dieta('rafael','highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.09879605588526212
              x: [ 8.607e-02  0.000e+00 ...  0.000e+00  1.273e-02]
            nit: 3
          lower:  residual: [ 8.607e-02  0.000e+00 ...  0.000e+00
                              1.273e-02]
                 marginals: [ 0.000e+00  4.093e-01 ...  4.399e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.823e-01  2.625e+01  4.148e+00
                              0.000e+00]
                 marginals: [-1.387e-02 -0.000e+00 -0.000e+00 -0.000e+00
                             -8.616e-04]
 mip_node_count: 0
 mip_dual_bound: 0

In [87]:
dieta('lucas','highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.15733041575492343
              x: [ 2.047e-02  0.000e+00 ...  0.000e+00  1.369e-01]
            nit: 2
          lower:  residual: [ 2.047e-02  0.000e+00 ...  0.000e+00
                              1.369e-01]
                 marginals: [ 0.000e+00  1.937e-01 ...  2.867e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  7.245e+00  1.296e+00
                              2.301e+00]
                 marginals: [-2.188e-02 -1.094e-02 -0.000e+00 -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0